In [1]:
import pandas as pd

In [2]:
finance = pd.read_csv('../data/step1/finance.csv', index_col=0)
returns = pd.read_csv('../data/step1/returns.csv', index_col=0)
for df in [finance, returns]:
    df['Time'] = pd.to_datetime(df['Time'])
d = pd.read_csv('../data/step3/Weighted_Return_Rate.csv', index_col=0)
d = dict(d[['label', 'R']].values)

In [3]:
T = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
for t in T:
    m = 7
    idx = []
    rm = []
    SMB = []
    HML = []
    RMW = []
    CMA = []
    rf = []
    for y in [t - 2, t - 1, t]:
        while (y < t and m <= 12) or (y == t and m <= 7):
            label = str(t) + '_' + str(y) + str(m)
            idx.append(label)
            rm.append(returns[(returns['Time'].dt.year == y) & (returns['Time'].dt.month == m)]['Cmretwdos'].iloc[0])
            SMB.append((d[label + '_BM_Aa'] + d[label + '_BM_Ab'] + d[label + '_BM_Ac'] + \
                        d[label + '_OP_Aa'] + d[label + '_OP_Ab'] + d[label + '_OP_Ac'] + \
                        d[label + '_INV_Aa'] + d[label + '_INV_Ab'] + d[label + '_INV_Ac'] - \
                        d[label + '_BM_Ba'] + d[label + '_BM_Bb'] + d[label + '_BM_Bc'] - \
                        d[label + '_OP_Ba'] + d[label + '_OP_Bb'] + d[label + '_OP_Bc'] - \
                        d[label + '_INV_Ba'] + d[label + '_INV_Bb'] + d[label + '_INV_Bc']) / 3)
            HML.append((d[label + '_BM_Aa'] + d[label + '_BM_Ba'] - d[label + '_BM_Ac'] - d[label + '_BM_Bc']) / 2)
            RMW.append((d[label + '_OP_Aa'] + d[label + '_OP_Ba'] - d[label + '_OP_Ac'] - d[label + '_OP_Bc']) / 2)
            CMA.append((d[label + '_INV_Aa'] + d[label + '_INV_Ba'] - d[label + '_INV_Ac'] - d[label + '_INV_Bc']) / 2)
            rf.append(returns[(returns['Time'].dt.year == y) & (returns['Time'].dt.month == m)]['Nrrmtdt'].iloc[0])
            m += 1
        m = 1
    df = pd.DataFrame([idx, rm, SMB, HML, RMW, CMA, rf], index=None).T
    df.columns = ['label', 'rm', 'SMB', 'HML', 'RMW', 'CMA', 'rf']
    df['rm-rf'] = df['rm'] - df['rf']
    df.to_csv('../data/step3/{}.csv'.format(t))